# Summarize Agent

- [summarize_agent.py](https://github.com/ks6088ts-labs/workshop-llm-agents/blob/main/workshop_llm_agents/agents/summarize_agent.py)

## References

- [How to summarize text through parallelization](https://python.langchain.com/docs/how_to/summarize_map_reduce/)


In [1]:
# Set up logging
import logging

# logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [2]:
# Load environment variables from .env
from dotenv import load_dotenv

assert load_dotenv(override=True), "Failed to load .env file"

In [3]:
# Set up LLM
from workshop_llm_agents.llms.azure_openai import AzureOpenAIWrapper

azure_openai_wrapper = AzureOpenAIWrapper()
llm = azure_openai_wrapper.get_azure_chat_openai()

In [4]:
# Load documents
from langchain_community.document_loaders import WebBaseLoader

web_path = "https://learn.microsoft.com/ja-jp/azure/ai-services/openai/overview"
loader = WebBaseLoader(
    web_path=web_path,
    header_template={
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",  # noqa: E501
    },
)

docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
# Split into chunks
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=3000,
    chunk_overlap=0,
)
chunks = text_splitter.split_documents(docs)
print(f"# of chunks: {len(chunks)}")

# of chunks: 3


In [6]:
# Agent
from workshop_llm_agents.agents.summarize_agent import SummarizeAgent

agent = SummarizeAgent(
    llm=llm,
)

In [7]:
# Dump an image of agent graph
png = "../docs/images/summarize_agent.png"
with open(png, "wb") as f:
    f.write(agent.mermaid_png(output_file_path=png))

In [8]:
import asyncio

response = await agent.run(docs=chunks)
print(f"final output: {response}")

['generate_summary']
['generate_summary']
['generate_summary']
['collect_summaries']
['generate_final_summary']
final output: The Azure OpenAI Service is a comprehensive platform that provides access to OpenAI's advanced language models, such as GPT-4o, GPT-4 Turbo with Vision, and GPT-3.5-Turbo, through REST API, Python SDK, and Azure AI Foundry. These models can be fine-tuned for tasks like content generation, summarization, image interpretation, and code translation. The service emphasizes responsible AI use with content filtering and ethical guidelines. Users must create an Azure OpenAI Service resource to deploy models and can utilize features like real-time audio and assistant capabilities. The service integrates OpenAI models with Azure's security features, supporting virtual networks and managed identities. Text and image processing involve tokenization, with costs varying by detail and size. Additionally, the service offers models like DALL-E for image generation and Whisper f